In [404]:
import subprocess
import json
import os
import time

from qdrant_client import QdrantClient

def reco(positives=[], negatives=[], top=10):
    positives_cli = []
    negatives_cli = []
    
    if len(positives) > 0:
        positives_cli = ['--positives', ','.join([str(p) for p in positives])]
    
    if len(negatives) > 0:
        negatives_cli = ['--negatives', ','.join([str(n) for n in negatives])]
        
    result = subprocess.run([
        'target/debug/scorer-playground',
        # Add positive and negative ids here
        *positives_cli,
        *negatives_cli,
        '--top', str(top),
        ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    
    graph_scored_points = json.loads(result)
    
    result = subprocess.run([
        'target/debug/scorer-playground',
        # Add positive and negative ids here
        *positives_cli,
        *negatives_cli,
        '--full-scan',
        '--top', str(top),
        ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    
    plain_scored_points = json.loads(result)
    
    # turn qdrant on
    os.system("cd ../.. && $(target/debug/qdrant) &")
    
    time.sleep(3)
    
    print('qdrant active on the background')
    
    qdrant_client = QdrantClient("localhost", port=6333)
    
    client_scored_points = qdrant_client.recommend(
        collection_name="food-plain",
        negative=negatives,
        positive=positives,
        limit=top,
    )
    
    # turn qdrant off
    !kill -9 $(pidof qdrant)
    print('qdrant killed')
    
    
    
    return (graph_scored_points, plain_scored_points, 
            client_scored_points
            )

In [405]:
(graph, plain, 
 client
 ) = reco(
    positives=[17,12557],
    negatives=[1111, 11258, 8350, 12728, 3970,2579, 10372, 11877,11517,12317,2166,8141],
    top=10,
)

#2342

qdrant active on the background
qdrant killed


sh: _: command not found


In [406]:
from IPython.display import Image, HTML

display(HTML(f"""
<div style='display:flex;'>
    <div style="margin:40px">
        <h1>Custom reco (graph)</h1><img  width=300 height=0>
    </div>
    <div style="margin:40px">
        <h1>Custom reco (full-scan)</h1><img  width=300 height=0>
    </div>
    <div style="margin:40px">
        <h1>Current reco</h1><img  width=300 height=0>
    </div>
</div>
"""))

for i, (graph_point, plain_point, client_point) in enumerate(zip(graph, plain, client)):
    graph_id = graph_point["id"]
    graph_score = (graph_point["score"])
    graph_image = (graph_point["payload"]['image'])
    
    plain_id = plain_point["id"]
    plain_score = (plain_point["score"])
    plain_image = (plain_point["payload"]['image'])
    
    client_id = client_point.id
    client_score = client_point.score
    client_image = client_point.payload['image']
    
    display(HTML(f"""
<div style='display:flex;'>
    <p>#{i+1}</p>
    <div style="margin:40px">
        <p>id: {graph_id}, score: {graph_score}</p><img src={graph_image} width=300>
    </div>
    <div style="margin:40px">
       <p>id: {plain_id}, score: {plain_score}</p><img src={plain_image} width=300>
    </div>
    <div style="margin:40px">
       <p>id: {client_id}, score: {client_score}</p><img src={client_image} width=300>
    </div>
</div>
    """))